Создаем представление, в котором собраны кастомеры, количество покупок, возвратов, сгруппированные по week_id, customer_id, item_id. Выбираю week_id с учетом года, т.к. недель в году 52, добавляю к номеру года два разряда для хранения номера недели. 

In [ ]:
CREATE OR REPLACE VIEW view staging.customers_count as (

        select week_id, item_id, customer_id, sum(case 
        when status = 'shipped' then 1
        else 0
        end) as shipped_qty, 
        sum(case 
        when status = 'refunded' then 1
        else 0
        end) as refunded_qty,
        sum(case
        when status = 'refunded' then payment_amount
        else 0
        end) as refunded,
        sum(case 
            when status = 'shipped' then payment_amount
            else 0
        end) as payment_amount
        from (
        select 
        s.customer_id, s.payment_amount, s.item_id, 
        100*c.year_actual+c.week_of_year as week_id, 
        status
        from mart.f_sales s
        left join mart.d_calendar c
        on s.date_id = c.date_id) all_cust
        group by week_id, customer_id, item_id);


Создаем представление для данных по новым клиентам, для этого забираем данные из вспомогательной таблицы с фильтром по количеству покупок строго равным одному. 

In [ ]:
CREATE or replace VIEW mart.f_new_customers as
select week_id, item_id, count(customer_id) as new_customers_count, sum(payment_amount) as new_customers_revenue from staging.customers_count where shipped_qty = 1
group by week_id, item_id;


Создаем представление для данных по повторным клиентам, для этого забираем данные из вспомогательной таблицы с фильтром по количеству покупок больше одного. 

In [ ]:
CREATE or replace VIEW mart.f_returning_customers as
select week_id, item_id, count(customer_id) as returning_customers_count, sum(payment_amount) as returning_customers_revenue from staging.customers_count where shipped_qty > 1
group by week_id, item_id;

Создаем представление для данных по возвратам, для этого забираем данные из вспомогательной таблицы с фильтром по количеству возвратов больше одного. В условиях не оговорено, нужно ли делать отделять клиентов с возвратами от всех остальных, поэтому эти группы могут пересекаться. 

In [ ]:
CREATE or replace VIEW mart.f_refunded_customers as
select week_id, item_id, count(customer_id) as refunded_customers_count, sum(payment_amount) as customers_refunded from staging.customers_count where refunded_qty > 0
group by week_id, item_id;

Объединяем данные из представлений в витрину mart.f_customer_retention. Сначала создадим таблицу mart.f_customer_retention.

In [ ]:
create table if not exists mart.f_customer_retention (
new_customers_count bigint default 0,
returning_customers_count bigint default 0,
refunded_customers_count bigint default 0,
period_name varchar(7) default 'weekly',
period_id bigint,
item_id int4,
new_customers_revenue numeric(10,2) default 0,
returning_customers_revenue numeric (10,2) default 0,
customers_refunded bigint default 0
);

А потом заполним её данными. Чтобы данные не дублировались и не терялись, объединяем по уникальным week_id и item_id из таблицы фактов f_sales. В некоторых строчках из-за этого возникнут нулевые значения. 

In [ ]:
with wi as (select 
distinct 100*c.year_actual+c.week_of_year as week_id, item_id
from mart.f_sales s
left join mart.d_calendar c
on s.date_id = c.date_id)
insert into  mart.f_customer_retention 
(new_customers_count, returning_customers_count, refunded_customers_count, 
period_id, item_id, 
new_customers_revenue, returning_customers_revenue, customers_refunded)
(select nc.new_customers_count, rc.returning_customers_count, rfc.refunded_customers_count, wi.week_id as period_id, wi.item_id, 
nc.new_customers_revenue, rc.returning_customers_revenue, rfc.customers_refunded
from wi left join mart.f_new_customers nc 
on wi.week_id = nc.week_id and wi.item_id = nc.item_id
left join mart.f_returning_customers rc
on wi.week_id = rc.week_id and wi.item_id = rc.item_id
left join mart.f_refunded_customers rfc
on wi.week_id = rfc.week_id and wi.item_id = rfc.item_id);